In [22]:
import cv2
import numpy as np
import sys

In [23]:
class Image:
    def __init__(self,name):
        def resize_image(name):
            image = cv2.imread(name,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(int(image.shape[1]*600/3024),int(image.shape[0]*800/4032)))
            return image
        def calculate(self):
                detector = cv2.AKAZE_create()
                keypoints, descriptors = detector.detectAndCompute(self.image,None)
                return keypoints, descriptors

        self.name = name
        self.image = resize_image(name)
        self.kp,self.des = calculate(self)
        
        def show(self):
            cv2.imshow(self.name, self.image)
        

In [24]:
def make_panorama(original1,original2):
    matcher = cv2.BFMatcher(cv2.NORM_L2,False)
    matches = matcher.knnMatch(original1.des,original2.des,2)
    goodmatches = []
    trainkeys = []
    querykeys = []
    maskArray = []

    for i in matches:
        print(i[0].distance,end='')
        if i[0].distance < 500:
            if i[0].distance/i[1].distance < 0.8:
                print('-->good match')
                goodmatches.append(i[0])
                querykeys.append((original1.kp[i[0].queryIdx].pt[0],original1.kp[i[0].queryIdx].pt[1]))
                trainkeys.append((original2.kp[i[0].trainIdx].pt[0],original2.kp[i[0].trainIdx].pt[1]))
            else:
                print('-->not good match')
        else:
            print('-->not good match')

    print("-----Calculating Homography-----")
    H, status = cv2.findHomography(numpy.array(trainkeys),numpy.array(querykeys),cv2.RANSAC)

    div = calcDst4(H,original2.image.shape,original1.image.shape)

    mask=numpy.zeros((height, width,1),numpy.uint8)
    panorama = cv2.warpPerspective(original2.image,H,(width,height))
    for i in range(width):
      for j in range(height):
        if original1.image[j,i].all():
          if panorama[j,i].all():
            panorama[j,i] = original1.image[j,i]/2+panorama[j,i]/2
            mask[j,i]=255
          else:
            panorama[j,i] = original1.image[j,i]
    #cv2.imwrite("mask",mask)
    cv2.waitKey(0)
    print("--next--")
    return panorama


In [25]:
if __name__=="__main__":
    image_names = sys.argv
    if len(image_names) == 1:
        print("usage: python3 panorama.py image1 image2 image3 ..")
        exit()

    if len(image_names) == 2:
        print("usage: ２枚以上画像のパスを渡してください")
        exit()

    print("--start--")
    images = []
    panorama = []
    for i in image_names[1:]:
        print(i)
        images.append(Image(i))

    panorama.append(WideImage(images[0].image))

    for i in range(0,len(images)-1):
      panorama.append(WideImage(make_panorama(panorama[i],images[i+1])))

    cv2.imwrite("panorama.png",panorama[-1].image)

    #for i in range(0,len(panorama)):
    #   cv2.imshow("result"+str(i),panorama[i].image)

    print("--FIN--")
    cv2.waitKey(0)
    

--start--
-f


AttributeError: 'NoneType' object has no attribute 'shape'